In [136]:
# # import all modules
import shutil
import glob
import csv
import ast
import os
import re

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D


import pandas as pd
import numpy as np
import math
import pymatgen.core as mg
import pymatgen.symmetry.analyzer as amg



# Calculate Rotational Symmetry Number with pymatgen

In [131]:
def get_symmetry_number(file):
    mol = amg.Molecule.from_file(file)
    return amg.PointGroupAnalyzer(mol).get_rotational_symmetry_number()

# Add column for Rotor Type to Datafile

In [137]:
df = pd.read_csv('/Users/z5380625/Documents/Research/4_BigData/B971_def2TZVPD_DataFile1.csv') # load csv file 
rays_asymm = (2*df['B [cm-1]'].values - df['A [cm-1]'].values - df['C [cm-1]'].values)/(df['A [cm-1]'].values - df['C [cm-1]'].values)
df['rays_asymm'] = rays_asymm

In [138]:
df

,Formula,SMILES,# Atoms,Conf#,Rotamer Degeneracy,Relative Energy [kJ/mol],BW Percent at 10 K,BW Percent at 50 K,BW Percent at 100 K,BW Percent at 298.15 K,...,Intensity,mu_a [D],mu_b [D],mu_c [D],mu_tot [D],mu_amu_bmu_c,A [cm-1],B [cm-1],C [cm-1],rays_asymm
0,C2H2,C#C,4,CONF1,1,0.000000,100.0,100.00,100.00,100.00,...,"[0.0, 0.0, 99.8956, 99.8956, 0.0, 93.1466, 0.0]",0.000000,0.000003,-0.000004,0.000005,-0.000000e+00,0.000000,1.184735,1.184735,-1.000000
1,C2H2O2,O=CC=O,6,CONF1,1,0.000000,100.0,100.00,100.00,100.00,...,"[33.6996, 50.3668, 0.0, 2.111, 0.0, 0.0, 8.127...",0.000002,-0.000001,0.000007,0.000007,-1.447200e-17,1.861842,0.159363,0.146798,-0.985347
2,C2H2OS_1,C#CSO,6,CONF1,1,0.000000,100.0,100.00,100.00,100.00,...,"[1.9386, 38.806, 53.7728, 20.2899, 68.4295, 9....",-0.548698,0.572922,-1.356004,1.571005,4.262752e-01,0.672851,0.131892,0.111405,-0.927021
3,C2H2OS_2,O=CC=S,6,CONF1,1,0.000000,100.0,100.00,100.00,100.00,...,"[16.4434, 17.5575, 5.9577, 18.578, 2.8034, 48....",-1.118671,0.091088,0.000076,1.122373,-7.764606e-06,1.725695,0.095213,0.090234,-0.993912
4,C2H2O_1,C1=CO1,5,CONF1,1,0.000000,100.0,100.00,100.00,100.00,...,"[0.4926, 81.8028, 16.8026, 33.8337, 70.4639, 1...",-2.078053,-1.057273,-0.000108,2.331551,-2.381622e-04,1.221050,0.800449,0.483497,-0.140530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10210,C6H8_6,CC1=CC1C=C,14,CONF2,3,3.827976,0.0,0.01,0.99,17.59,...,"[0.343, 0.0441, 3.148, 1.9534, 8.3146, 7.2842,...",-1.193794,0.308805,-0.072778,1.235234,2.682954e-02,0.219722,0.082840,0.074881,-0.890094
10211,C6H8_6,CC1=CC1C=C,14,CONF1,3,0.000000,100.0,99.99,99.01,82.41,...,"[0.4077, 0.0324, 2.6933, 1.6031, 5.9492, 9.959...",-1.438531,0.208010,-0.132882,1.459553,3.976191e-02,0.276880,0.070069,0.060904,-0.915128
10212,C6H8_8,CC1=CC=CC1,14,CONF1,3,0.000000,100.0,100.00,99.90,91.07,...,"[0.0746, 0.441, 0.7913, 13.283, 0.273, 0.3618,...",-0.563956,-0.479358,0.000000,0.740155,0.000000e+00,0.268572,0.107810,0.079179,-0.697651
10213,C6H8_8,CC1=CC=CC1,14,CONF2,2,5.755353,0.0,0.00,0.10,8.93,...,"[0.0801, 0.4321, 0.7899, 13.2812, 0.2709, 0.36...",-0.563954,-0.479296,0.000151,0.740114,4.089656e-05,0.268571,0.107810,0.079179,-0.697649


In [139]:
formulas = df['Formula'].values
num_atoms = df['# Atoms'].values
confs = df['Conf#'].values

formula_conf_list = list(zip(formulas,confs,num_atoms))
sigma_list = []
for formula_conf in formula_conf_list:
    formula = formula_conf[0]
    conf = formula_conf[1]
    num = formula_conf[2]
    if num <=6:
        folder_loc ='6_Atoms'
    elif (num>6) & (num<=8):
        folder_loc = '8_Atoms'
    else:
        folder_loc = str(num)+'_Atoms'
    file = '/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/4_VibrationalCalcs/FinalGeoms_VibCalc/'+formula+'_'+conf+'_VibCalcGeom.xyz'
    sigma = get_symmetry_number(file)
    sigma_list.append(sigma)
    


In [140]:
df['Sigma'] = sigma_list

In [141]:
A=df['A [cm-1]'].values
B=df['B [cm-1]'].values
C=df['C [cm-1]'].values
sigma=df['Sigma'].values
ABC_list = list(zip(A,B,C,sigma))

rotor_type_list = []
qrot_list = []
for ABC in ABC_list:
    A = ABC[0]
    B = ABC[1]
    C = ABC[2]
    sigma = ABC[3]
    rotor_type = define_mol_rotor_type(A,B,C)
    A = ABC[0]*29979.2458
    B = ABC[1]*29979.2458
    C = ABC[2]*29979.2458
    if rotor_type == 'Spherical':
        rotor_type_list.append('Spherical')
        qrot = qrot_asym(A,B,C,sigma)
        qrot_list.append(qrot)
    elif rotor_type == 'Linear':
        rotor_type_list.append('Linear')
        qrot = qrot_linear(B)
        qrot_list.append(qrot)
    elif rotor_type == 'Quasilinear':
        rotor_type_list.append('Quasilinear')
        qrot = qrot_linear(B)
        qrot_list.append(qrot)
    elif rotor_type == 'Prolate':
        rotor_type_list.append('Prolate')
        qrot = qrot_prolate(A,B,sigma)
        qrot_list.append(qrot)
    elif rotor_type == 'Oblate':
        rotor_type_list.append('Oblate')
        qrot = qrot_oblate(B,C,sigma)
        qrot_list.append(qrot)
    elif rotor_type == 'Asymmetric':
        rotor_type_list.append('Asymmetric')
        qrot = qrot_asym(A,B,C,sigma)
        qrot_list.append(qrot)


df['Rotor Type'] = rotor_type_list
df['Qrot @ 9.375 K'] = qrot_list


22.227575125275504
28.199194187206672
20.01218360372495
17.19812209160045


In [ ]:
df[df['Rotor Type']=='Quasilinear']

,Formula,SMILES,# Atoms,Conf#,Rotamer Degeneracy,Relative Energy [kJ/mol],BW Percent at 10 K,BW Percent at 50 K,BW Percent at 100 K,BW Percent at 298.15 K,...,mu_c [D],mu_tot [D],mu_amu_bmu_c,A [cm-1],B [cm-1],C [cm-1],rays_asymm,Sigma,Rotor Type,Qrot @ 9.375 K
57,CHNO_1,OC#N,4,CONF1,1,0.0,100.0,100.0,100.0,100.0,...,0.000000e+00,3.956787,0.000000e+00,22.582532,0.354957,0.349464,-0.999506,1,Quasilinear,18.357339
58,CHNO_2,N=C=O,4,CONF1,1,0.0,100.0,100.0,100.0,100.0,...,-2.000000e-07,2.060141,-4.209254e-07,28.568893,0.369699,0.364976,-0.999665,1,Quasilinear,17.625325
85,HN3,N=[N+]=[N-],4,CONF1,1,0.0,100.0,100.0,100.0,100.0,...,-4.000000e-07,1.755340,5.553433e-07,20.416896,0.404712,0.396846,-0.999214,1,Quasilinear,16.100493
86,HNO,N=O,3,CONF1,1,0.0,100.0,100.0,100.0,100.0,...,0.000000e+00,1.631487,-0.000000e+00,18.632742,1.434620,1.332059,-0.988144,1,Quasilinear,4.542018


In [142]:
df[df['Formula']=='C2N2O']

,Formula,SMILES,# Atoms,Conf#,Rotamer Degeneracy,Relative Energy [kJ/mol],BW Percent at 10 K,BW Percent at 50 K,BW Percent at 100 K,BW Percent at 298.15 K,...,mu_c [D],mu_tot [D],mu_amu_bmu_c,A [cm-1],B [cm-1],C [cm-1],rays_asymm,Sigma,Rotor Type,Qrot @ 9.375 K
19,C2N2O,N#CN=C=O,5,CONF1,1,0.0,100.0,100.0,100.0,100.0,...,3.000000e-07,2.62281,3.445377e-07,3.291466,0.087287,0.085032,-0.998593,1,Asymmetric,188.617856


# Define Molecular Rotor Type

In [ ]:
def define_mol_rotor_type(A,B,C):
    # spherical
    if math.isclose(A,B,rel_tol=1e-2) and math.isclose(B,C,rel_tol=1e-2):
        return 'Spherical'
    # linear
    elif math.isclose(A,0.00,rel_tol=1e-2) and math.isclose(B,C,rel_tol=1e-2):
        return 'Linear'
    elif math.isclose(B,C,rel_tol=1e-1) and A-B>10: #10 cm-1
        print(A-B)
        return 'Quasilinear'
    # prolate
    elif math.isclose(B,C,rel_tol=1e-2) and A>B and A!=0.:
        return 'Prolate'
    # oblate
    elif math.isclose(A,B,rel_tol=1e-2) and B>C and C!=0.:
        return 'Oblate'
    # asymmetric
    else:
        return 'Asymmetric'
    

# Calculate Qrot at 9.375 K for all Molecules

In [ ]:
def qrot_linear(B):
    return 2.0837e4*(9.375/B)

def qrot_prolate(A,B,sigma):
    return 5.3311e6*(9.375**3/(B**2*A))**0.5/sigma

def qrot_oblate(B,C,sigma):
    return 5.3311e6*(9.375**3/(B**2*C))**0.5/sigma

def qrot_asym(A,B,C,sigma):
    return 5.3311e6*(9.375**3/(A*B*C))**0.5/sigma

# Create .var files for SPCAT

line 1: title 
line 2 [freeform]: NPAR, NLINE, NITR, NXPAR, THRESH , ERRTST, FRAC, CAL
(only NPAR used by SPCAT)
line 3 option information[freeform]:
CHR, SPIND, NVIB, KNMIN, KNMAX, IXX, IAX, WTPL, WTMN, VSYM, EWT, DIAG, XOPT

In [ ]:
def make_spcat_var(df,folder_loc,file):
    filen = file.split('/')[-1].split('_harmonic')[0]
    mol = file.split('/')[-1].split('_harmonic')[0].split('_')
    conf = mol[-1]
    if len(mol) == 3:
        mol = mol[0]+'_'+mol[1]
    else:
        mol = mol[0]

    with open(file,'r') as inp:
        lines = inp.readlines()
        indices = [i for i, x in enumerate(lines) if x == '--------------------------------------------------------------------------------\n']
        spcat_var_list = lines[indices[-2]+1:indices[-1]]

    inp.close()
    
    rotor_type = df[(df['Formula']==mol)&(df['Conf#']==conf)]['Rotor Type'].values[0]
    if (rotor_type == 'Linear') or (rotor_type == 'Quasilinear'):
        kmax = 0
    else:
        kmax = 99
    spcat_var_list[2] = 'a 00 1 0 '+str(kmax)+'\n'


    with open('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/5_SPCAT/InputFiles/'+filen+'.var','a') as f:
        for line in spcat_var_list:
            f.write(line)
    f.close()
    return
            

In [ ]:
folder_loc_list = ['6_Atoms','8_Atoms','9_Atoms','10_Atoms','11_Atoms','12_Atoms','13_Atoms','14_Atoms','15_Atoms','16_Atoms','17_Atoms','18_Atoms','19_Atoms','20_Atoms']
for folder_loc in folder_loc_list:
    file_names = glob.glob('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/4_VibrationalCalcs/OutputFiles/*.log')
    # clear folder
    files_spcat = glob.glob('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/5_SPCAT/InputFiles/*')
    for f in files_spcat:
        os.remove(f)
    
    for file in file_names:
        make_spcat_var(df,folder_loc,file)

# Create .int files for SPCAT


line 1:title
line 2 [freeform]: FLAGS,TAG,QROT,FBGN,FEND,STR0,STR1,FQLIM,TEMP,MAXV
line 3-end [freeform]: IDIP,DIPOLE

In [ ]:
def get_dipole_moments(file):
    dipole_info_all = []
    with open(file,'r+') as inp:
        for line in inp:
            line = line.strip()
            if line.startswith('Dipole moment (Debye):'):
                # print(next(inp).split())
                dipole_info_all.append(next(inp).split())
    inp.close()
    row = dipole_info_all[-1]
    mu_a, mu_b, mu_c = float(row[0]),float(row[1]),float(row[2])
    return mu_a, mu_b, mu_c


In [ ]:
def make_spcat_int(df,folder_loc,file,counter):
    mol = file.split('/')[-1].split('_harmonic')[0].split('_')
    conf = mol[-1]
    if len(mol) == 3:
        mol = mol[0]+'_'+mol[1]
    else:
        mol = mol[0]
    
    filen = file.split('/')[-1].split('_harmonic')[0]
    identifier = '{0:0>4}'.format(counter)
    qrot = '{0:0.3f}'.format(df[(df['Formula']==mol)&(df['Conf#']==conf)]['Qrot @ 9.375 K'].values[0])
    mu_a, mu_b, mu_c = get_dipole_moments(file)


    with open('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/5_SPCAT/InputFiles/'+filen+'.int','a') as f:
        f.write(filen+'\n')
        f.write('112    '+str(identifier)+'  '+str(qrot)+'    0   20  -8. -8. 30000   9.375\n')
        f.write('        001    '+'{0:.7f}'.format(mu_a)+'                / a dipole /\n')
        f.write('        002    '+'{0:.7f}'.format(mu_b)+'                / b dipole /\n')
        f.write('        003    '+'{0:.7f}'.format(mu_c)+'                / c dipole /\n')
    f.close()
    return


In [129]:
folder_loc_list = ['6_Atoms','8_Atoms','9_Atoms','10_Atoms','11_Atoms','12_Atoms','13_Atoms','14_Atoms','15_Atoms','16_Atoms','17_Atoms','18_Atoms','19_Atoms','20_Atoms']


counter=1
for folder_loc in folder_loc_list:
    file_names = glob.glob('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/4_VibrationalCalcs/OutputFiles/*.log')
    for file in file_names:
        make_spcat_int(df,folder_loc,file,counter)
        counter+=1



 Vibro-Rot alpha Matrix (in MHz)
 -------------------------------
                 a              b              c
 Q( 1)     6193.47577     6206.71628     -741.88870
 Q( 2)     -450.39454    -5345.10507   776425.87899
 Q( 3)    -5326.66076     -445.67724  -774885.24623
 Q( 4)     -946.54466     -949.51591     -843.72967
 Q( 5)    -1398.62821     -975.53842    21922.01470
 Q( 6)     -977.13774    -1397.24702   -24078.58137